In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [2]:
sc = SparkContext()

In [3]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

In [4]:
socket_stream = ssc.socketTextStream("localhost", 5557)

In [5]:
lines = socket_stream.window( 20 )

In [6]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [7]:
#connexion à cassandra
def saveToCassandra(row):
    
    if not row.isEmpty(): 
        
        sqlContext.createDataFrame(row).write\
        .format("org.apache.spark.sql.cassandra")\
        .mode('append')\
        .options(table="tagcount", keyspace="ereputation")\
        .save()

In [8]:
( lines.flatMap( lambda text: text.split( " " ) )
  .filter( lambda word: word.lower().startswith("#") )
 .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") )
 #            .limit(10).registerTempTable("tweets") ) )
    .foreachRDD(lambda rdd:saveToCassandra))

In [9]:
# Start the computation
ssc.start()

In [10]:
#ssc.stop()